In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import cv2 as cv
import tensorflow as tf
import matplotlib.pyplot as plt
import glob
from matplotlib.image import imread
import tensorflow_datasets as tfds
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator


print("Pandas Version %s" %pd.__version__)
print("Numpy version %s" % np.__version__)
print("Tensorflow Version %s" % tf.__version__)
print("CV Version %s" % cv.__version__)

Pandas Version 1.1.5
Numpy version 1.19.5
Tensorflow Version 2.4.1
CV Version 4.1.2


In [3]:
TRAININGCT = "/content/drive/My Drive/XrayModel/DataSetCT/train/"

TESTCT = "/content/drive/My Drive/XrayModel/DataSetCT/test/"

strategy = tf.distribute.MirroredStrategy()
print("Number of devices %d" %(strategy.num_replicas_in_sync))
BATCH_PER_REPLICA = 32 
BATCH_SIZE = BATCH_PER_REPLICA * strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices 1


In [4]:
trainCtDatagen = ImageDataGenerator(rescale=  1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range= 0.2,
                                   shear_range = 0.2,
                                   zoom_range=0.2,
                                   horizontal_flip = True,
                                   fill_mode="nearest")
trainCtGenerator = trainCtDatagen.flow_from_directory(TRAININGCT,
                                                     batch_size=BATCH_SIZE,
                                                     class_mode="binary",
                                                     target_size=(150,150))
testCtDatagen = ImageDataGenerator(rescale=  1./255,)
testCtGenerator = testCtDatagen.flow_from_directory(TESTCT,
                                                   batch_size=BATCH_SIZE,
                                                   class_mode="binary",
                                                   target_size=(150,150))


Found 2481 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [33]:
stop = tf.keras.callbacks.EarlyStopping(patience=5)
with strategy.scope():
    
    inception=tf.keras.applications.inception_v3.InceptionV3(include_top=False,input_shape=(150,150,3),weights='imagenet')
    layers = inception.get_layer("mixed7")
    finLayer = tf.keras.layers.Flatten()(layers.output)
        
    dense = tf.keras.layers.Dense(units=256,activation=tf.nn.relu)(finLayer)
    dense = tf.keras.layers.Dropout(0.3)(dense)
    dense = tf.keras.layers.Dense(units=128,activation=tf.nn.relu)(dense)
    dense = tf.keras.layers.Dropout(0.4)(dense)
    dense = tf.keras.layers.Dense(units=64, activation=tf.nn.relu)(dense)
    output = tf.keras.layers.Dense(units=1,activation=tf.nn.sigmoid)(dense)
    ctScanModel= tf.keras.Model(inputs=inception.input,outputs= output)

ctScanModel.compile(optimizer = tf.keras.optimizers.Adam(lr=0.0001),
                 loss ="binary_crossentropy", 
                 metrics=["acc"])
history = ctScanModel.fit(trainCtGenerator,epochs=15,
                       validation_data=testCtGenerator,callbacks=[stop])

Epoch 1/15
78/78 [==============================] - 44s 378ms/step - loss: 0.6146 - acc: 0.6539 - val_loss: 0.4930 - val_acc: 0.7000
Epoch 2/15
78/78 [==============================] - 27s 350ms/step - loss: 0.3558 - acc: 0.8574 - val_loss: 1.0479 - val_acc: 0.5950
Epoch 3/15
78/78 [==============================] - 27s 350ms/step - loss: 0.2573 - acc: 0.9085 - val_loss: 0.4092 - val_acc: 0.8150
Epoch 4/15
78/78 [==============================] - 27s 350ms/step - loss: 0.1939 - acc: 0.9244 - val_loss: 0.3364 - val_acc: 0.8450
Epoch 5/15
78/78 [==============================] - 27s 350ms/step - loss: 0.2009 - acc: 0.9121 - val_loss: 0.1657 - val_acc: 0.9350
Epoch 6/15
78/78 [==============================] - 27s 349ms/step - loss: 0.1992 - acc: 0.9237 - val_loss: 0.1824 - val_acc: 0.9300
Epoch 7/15
78/78 [==============================] - 27s 351ms/step - loss: 0.1595 - acc: 0.9393 - val_loss: 0.1448 - val_acc: 0.9350
Epoch 8/15
78/78 [==============================] - 27s 349ms/step - 

In [78]:
from tensorflow.keras.preprocessing import image
randCovImg = image.load_img(TRAININGCT + "/covid/"+"Covid (1).png",target_size=(150,150))
randCovImgArray =image.img_to_array(randCovImg)
randCovImgArray /= 255
ctScanModel.predict(tf.expand_dims(randCovImgArray,axis=0))

array([[0.00060031]], dtype=float32)

In [77]:
from tensorflow.keras.preprocessing import image
randNonCovImg = image.load_img(TRAININGCT + "/noncovid/"+"Non-Covid (1).png",target_size=(150,150,3))
randNonCovImgArray =image.img_to_array(randNonCovImg)
print(randNonCovImgArray.shape)
ctScanModel.predict(tf.expand_dims(randNonCovImgArray,axis=0))

(150, 150, 3)


array([[1.]], dtype=float32)

In [36]:
converter = tf.lite.TFLiteConverter.from_keras_model(ctScanModel) # Your model's name
model = converter.convert()
file = open( 'ctScan.tflite' , 'wb' ) 
file.write( model )

INFO:tensorflow:Assets written to: /tmp/tmp18fagq4h/assets


74566540